In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, Flatten, MaxPool2D, GlobalAveragePooling2D, BatchNormalization, ReLU, Dense, Input
from tensorflow.keras.models import Model
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [2]:
def train_dg(path):

    train = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    vertical_flip=True,
    rescale=1./255,
    )

    train = train.flow_from_directory(
    directory=path,
    batch_size=32,
    target_size=(224, 224),
    class_mode="categorical"
    )
    return train

In [3]:
def val_dg(path):

    test = ImageDataGenerator(
        rescale=1./255
    )

    test = test.flow_from_directory(
    directory=path,
    batch_size=32,
    target_size=(224, 224),
    class_mode="categorical"
    )
    return test

In [4]:
train_datagen = train_dg(r"E:\archive\data\train")

Found 2942 images belonging to 3 classes.


In [5]:
test_datagen = val_dg(r"E:\archive\data\val")

Found 737 images belonging to 3 classes.


In [6]:
def conv(x, filters, kernel_size, strides):
    x = Conv2D( filters = filters, kernel_size = kernel_size, strides= strides, padding='same', activation="relu", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

def depv(x, filters, kernel_size, strides):
    x = DepthwiseConv2D(kernel_size=kernel_size, strides=strides,  padding='same',use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters = filters, kernel_size = (1, 1), strides= 1, padding='same', activation="relu", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

In [7]:
def model(x):
    x = conv(x, 32, 3, 2)
    x = depv(x, 64, 3, 1)
    x = depv(x, 128, 3, 2)
    x = depv(x, 128, 3, 1)
    x = depv(x, 256, 3, 2)
    x = depv(x, 256, 3, 1)
    x = depv(x, 512, 3, 2)
    for i in range(5):
        x = depv(x, 512, 3, 1)
    x = depv(x, 1024, 3, 2)
    x = depv(x, 1024, 3, 2)
    # x = Flatten()(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(units=1024, activation="relu")(x)
    x = Dense(units=3, activation="softmax", use_bias=False)(x)
    return x

In [8]:
input = Input(shape=(224, 224, 3))
output = model(input)
cnn = Model(inputs=input, outputs=output)

In [9]:
cnn.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 112, 112, 32)      864       
                                                                 
 batch_normalization (Batch  (None, 112, 112, 32)      128       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 112, 112, 32)      0         
                                                                 
 depthwise_conv2d (Depthwis  (None, 112, 112, 32)      288       
 eConv2D)                                                        
                                                                 
 batch_normalization_1 (Bat  (None, 112, 112, 32)      128   

In [10]:
cnn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
callback = tf.keras.callbacks.EarlyStopping(monitor="accuracy", patience=3)

In [12]:
cnn.fit(x=train_datagen, validation_data=test_datagen, epochs=30, batch_size=32, callbacks=[callback])

Epoch 1/30


92/92 [==============================] - 250s 3s/step - loss: 0.6431 - accuracy: 0.7175 - val_loss: 1.1008 - val_accuracy: 0.3148
Epoch 2/30
92/92 [==============================] - 205s 2s/step - loss: 0.4351 - accuracy: 0.8253 - val_loss: 1.0991 - val_accuracy: 0.3148
Epoch 3/30
92/92 [==============================] - 200s 2s/step - loss: 0.3455 - accuracy: 0.8712 - val_loss: 1.0996 - val_accuracy: 0.3148
Epoch 4/30
92/92 [==============================] - 230s 3s/step - loss: 0.2671 - accuracy: 0.9062 - val_loss: 1.1715 - val_accuracy: 0.3066
Epoch 5/30
92/92 [==============================] - 220s 2s/step - loss: 0.2445 - accuracy: 0.9092 - val_loss: 2.1846 - val_accuracy: 0.3066
Epoch 6/30
92/92 [==============================] - 234s 3s/step - loss: 0.2273 - accuracy: 0.9283 - val_loss: 2.2198 - val_accuracy: 0.3066
Epoch 7/30
92/92 [==============================] - 256s 3s/step - loss: 0.2053 - accuracy: 0.9290 - val_loss: 0.5183 - val_accuracy: 0.7693
Epoch 8/30
